# Import

In [ ]:
!git clone https://github.com/Edouard99/Idemia.git

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch
import torch.nn
import numpy as np 
import struct
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from Idemia.facenet_model import *
from Idemia.utils import *
import torch.optim as optim
import tqdm
import os
from google.colab import drive
from google.colab import files
drive.mount("/content/gdrive", force_remount=True)

In [3]:
!cp /content/gdrive/MyDrive/Idemia/db_train.raw /content/
!cp /content/gdrive/MyDrive/Idemia/db_val.raw /content/
!cp /content/gdrive/MyDrive/Idemia/label_train.txt /content/
!mkdir /content/results/

# Init Data

In [ ]:
train_ds,val_ds=generate_datasets('./db_train.raw','./label_train.txt',0.7) #Generate Dataset from data, label and dividing the data according to a ratio
dataloader_t = torch.utils.data.DataLoader(train_ds,batch_size=32,shuffle=True,drop_last=False) #Dataloader train set
dataloader_v = torch.utils.data.DataLoader(val_ds,batch_size=32,shuffle=True,drop_last=False) #Dataloader validation set

# Model

In [15]:
device= torch.device("cuda:0")
facenet=FaceNet(0.2,0.7,True).to(device) #Creates the model

In [16]:
optimizer = optim.Adam(facenet.parameters(), lr=3e-4, betas=(0.9, 0.999)) #Adam Optimizer 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.3,verbose=True) #Learning rate scheduler
alpha1=0.2 #weight of the loss of the first intermediate output of the net used
alpha2=0.2 #weight of the loss of the second intermediate output of the net used
num_epochs=26
threshold=0.5

In [ ]:
training(num_epochs,facenet,optimizer,scheduler,dataloader_t,dataloader_v,device,alpha1,alpha2,threshold,"/content/results/")

In [ ]:
!zip -q /content/results_1.zip -r /content/results
!cp /content/results_1.zip /content/gdrive/MyDrive/Idemia/

# Test

In [ ]:
""" This cell can generate a dataset from data path & label path"""

test_ds=generate_datasets_test(path_images,path_label)
dataloader_test = torch.utils.data.DataLoader(test_ds,batch_size=32,shuffle=True,drop_last=False) #Dataloader train set

In [ ]:
""" This cell creates the model for testing and initialize the model with a defined checkpoint (weights of the model)"""
device= torch.device("cuda:0")
checkpoint=torch.load("")
facenet=create_test_model(0.2,0.7,device,checkpoint)

In [ ]:
""" Compute the prediction on the dataloader on test set and compute the Hter score of the set"""
threshold=0.5
tp,tn,fp,fn=0,0,0,0
for i, dataj in enumerate(dataloader_test, 0): #Evaluation on validation set
    x=dataj[0].float().to(device)
    gt=dataj[1].float().to(device)
    y=facenet(x).view(-1)
    pred=y>threshold
    tp+=torch.sum(((pred)==gt)[torch.where((gt)==1)]).item()#TP
    fn+=torch.sum(((pred)!=gt)[torch.where((gt)==1)]).item()#FN
    tn+=torch.sum(((pred)==gt)[torch.where((gt)==0)]).item()#TN
    fp+=torch.sum(((pred)!=gt)[torch.where((gt)==0)]).item()#FP
far=fp/(tn+fp)
frr=fn/(tp+fn)
hter=0.5*(far+frr)
print("Hter : {} \n".format(hter))

# Test Prediction

In [ ]:
with open('/content/db_val.raw', 'rb') as f: #Loading Test images
    data = np.fromfile(f, dtype=np.dtype(np.uint8))
    data = data.reshape((-1,56, 56, 3))

In [ ]:
device= torch.device("cuda:0")
checkpoint=torch.load("/content/final.pth")
facenet=create_test_model(0.2,0.7,device,checkpoint) #Create the model with the weights of the epoch 18 (best epoch for validation value)

In [ ]:
""" Making Prediction on the data"""

threshold=0.5
output=[] #Raw output of the network
prediction=[] #0 or 1 depending if the output of the network is >1 or <1
for img in tqdm.tqdm(data):
    x=transforms.ToTensor()(img).unsqueeze(0).float().to(device)
    y=facenet(x).view(-1)
    pred=y>threshold
    output.append(y.item())
    prediction.append(pred.item())

In [ ]:
with open('/content/test_net_output.txt','w') as f: #Writing rawoutput in a txt file
    for y in output:
        f.write("{}\n".format(y))
    print('Done')

In [ ]:
with open('/content/test_labeled.txt','w') as f: #Writing predicted label of the data in a txt file
    for y in prediction:
        if y==True:
          f.write("1\n")
        else:
          f.write("0\n")
    print('Done')